# Clifford+RZ

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

In [2]:
import collections
import copy
import math
import os
import random
# import warnings

import numpy as np

import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc

from qiskit import QuantumCircuit
from qiskit.compiler import transpile

from pyqrack import QrackSimulator, QrackCircuit

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ["QRACK_MAX_CPU_QB"]="-1"

In [3]:
def normalize_2x2_unitary(mtrx):
    det = np.linalg.det(mtrx)
    a = (mtrx[0][0] + np.conj(mtrx[1][1] / det)) / 2
    b = (mtrx[0][1] + np.conj(-mtrx[1][0] / det)) / 2
    return np.array([[a, b], [-det * np.conj(b), det * np.conj(a)]], dtype=np.complex128)

In [4]:
width = 6
max_magic = 6

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [5]:
def rand_1qb(sim, q):
    # th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    # lm = random.uniform(0, 4 * math.pi)
    # sim.u(th, ph, lm, q)
    sim.u(0, ph, 0, q)

def random_circuit(width, circ):
    t_count = 0
    single_bit_gates = circ.h, circ.x, circ.y
    two_bit_gates = circ.cx, circ.cy, circ.cz, circ.swap, circ.iswap

    for n in range(3 * width):
        # Single bit gates
        for j in range(width):
            random.choice(single_bit_gates)(j)
            if (t_count < max_magic) and ((width * width * random.random() / max_magic) < 1):
                rand_1qb(circ, j)
                t_count += 1

        # Multi bit gates
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            random.choice(two_bit_gates)(b1, b2)

    return circ

If we have Qiskit and numpy installed, we can convert to a Qiskit circuit. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In [6]:
orig_circ = random_circuit(width, QuantumCircuit(width))
qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(orig_circ, 0)
qsim.out_to_file("qrack_circuit.chp")
qsim.reset_all()
qsim = None

print("Raw (near-Clifford) gate count: ", sum(dict(orig_circ.count_ops()).values()))
print("Raw (near-Clifford) depth of critical path: ", orig_circ.depth())
print("Raw (near-Clifford) qubit width: ", orig_circ.width())

Raw (near-Clifford) gate count:  168
Raw (near-Clifford) depth of critical path:  40
Raw (near-Clifford) qubit width:  6


In [7]:
circ = QrackSimulator.file_to_optimized_qiskit_circuit("qrack_circuit.chp")

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())
print("(Near-Clifford) qubit width: ", circ.width())

     ┌────────────┐                    ┌─────────────────┐                   »
q_0: ┤ U(π/2,0,π) ├───────────■────────┤ U(π/2,π/2,-π/2) ├───────────────────»
     └────────────┘           │        └─────────────────┘                   »
q_1: ─────────────────────────┼──────────────────────────────────────────────»
                            ┌─┴─┐                                       ┌───┐»
q_2: ───────────────X───────┤ X ├───────────────■───────────────────────┤ X ├»
                    │       └───┘             ┌─┴─┐       ┌────────────┐└─┬─┘»
q_3: ───────────────┼─────────────────────────┤ X ├───────┤ U(π/2,0,π) ├──■──»
     ┌────────────┐ │ ┌───────────────┐       └───┘       └────────────┘     »
q_4: ┤ U(π/2,0,π) ├─X─┤ U(π/2,π/2,-π) ├──────────────────────────────────────»
     └────────────┘   └───────────────┘                                      »
q_5: ────────────────────────────────────────────────────────────────────────»
                                                    

`QrackCircuit` might reduce the gate count and depth further, but it's probably an unrealistic gate set for most gate-based quantum hardware. However, if we're using conventional Qrack simulation, rather than near-Clifford, this format is advantageous for native Qrack.

In [8]:
qcircuit = QrackCircuit.in_from_qiskit_circuit(circ)
qcircuit.out_to_file("qrack_circuit.qc")
qrack_circ = QrackCircuit.file_to_qiskit_circuit("qrack_circuit.qc")

if qrack_circ.width() <= 16:
    print(qrack_circ)
print("(Near-Clifford) gate count: ", sum(dict(qrack_circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", qrack_circ.depth())
print("(Near-Clifford) qubit width: ", qrack_circ.width())

                     ┌─────────────┐                                »
q_0: ────────────────┤ Multiplexer ├────────────────────────────────»
                     └─────────────┘                                »
q_1: ───────────────────────────────────────────────────────────────»
     ┌──────────────┐               ┌──────────────┐┌──────────────┐»
q_2: ┤1             ├───────────────┤0             ├┤1             ├»
     │              │┌─────────────┐│              ││              │»
q_3: ┤  Multiplexer ├┤ Multiplexer ├┤  Multiplexer ├┤  Multiplexer ├»
     │              │└─────────────┘│              ││              │»
q_4: ┤0             ├───────────────┤1             ├┤0             ├»
     └──────────────┘               └──────────────┘└──────────────┘»
q_5: ───────────────────────────────────────────────────────────────»
                                                                    »
«     ┌──────────────┐┌─────────────┐                ┌──────────────┐»
«q_0: ┤1           

We can further optimize by post-selecting with tensor network software.

In [9]:
tc.set_backend("tensorflow")
tc.set_contractor("auto")
tc.set_dtype("complex64")

net = tc.Circuit.from_qiskit(circ)
for b in range(width, circ.width()):
    net.post_select(b, keep=0)
net = tcsc.simple_compile(net)[0]

We can sample with tensorcircuit at this point, or we can try another round of optimization.

In [10]:
# %%time
# shots = 1 << (width + 12)
# print(net.sample(allow_state=True, batch=shots, format="count_dict_bin"))

In [11]:
def cross_entropy(measurement_list, qiskit_circuit):
    perm_count = 1 << qiskit_circuit.width()
    ideal_shots = perm_count << 12

    sim = QrackSimulator(qubitCount = qiskit_circuit.width(), isStabilizerHybrid=False)
    sim.run_qiskit_circuit(qiskit_circuit, 0)
    ideal_result = sim.measure_shots(list(range(qiskit_circuit.width())), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots
        
    ideal_probs_sv = []
    for i in range(perm_count):
        ideal_probs_sv.append(ideal_result[i] if i in ideal_result else 0)
    
    ideal_probs = []
    for bit_string in measurement_list:
        ideal_probs.append(ideal_probs_sv[bit_string])
                               
    return (perm_count * np.mean(ideal_probs)) - 1


def hellinger_fidelity(measurement_list, qiskit_circuit):
    perm_count = 1 << qiskit_circuit.width()
    ideal_shots = perm_count << 12

    t_histogram = collections.Counter(measurement_list)
    shot_count = sum(t_histogram.values())
    t_histogram = dict(t_histogram)
        
    histogram = {}
    for key, value in t_histogram.items():
        histogram[key] = value / shot_count

    sim = QrackSimulator(qubitCount = qiskit_circuit.width(), isStabilizerHybrid=False)
    sim.run_qiskit_circuit(qiskit_circuit, 0)
    ideal_result = sim.measure_shots(list(range(qiskit_circuit.width())), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots
        
    ideal_probs_sv = []
    for i in range(perm_count):
        ideal_probs_sv.append(ideal_result[i] if i in ideal_result else 0)

    fidelity = 0
    for qubit_permutation in histogram.keys():
        ideal_prob = ideal_result[qubit_permutation] if qubit_permutation in ideal_result else 0
        normalized_frequency = histogram[qubit_permutation]
        fidelity += math.sqrt(ideal_prob * ideal_prob)
    fidelity *= fidelity
                               
    return fidelity


if width <= 16:
    # The circuit has 0 ancillae, so we can generate samples with conventional simulation.
    qsim = QrackSimulator()
    qcircuit.run(qsim)
    for i in range(width, qcircuit.get_qubit_count()):
        qsim.force_m(i, False)
    shots = 1 << (width + 12)
    samples = qsim.measure_shots(list(range(width)), shots)
    # print("XEB fidelity: ", cross_entropy(samples, orig_circ))
    print("Hellinger fidelity: ", hellinger_fidelity(samples, orig_circ))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Hellinger fidelity:  1.0
